In [ ]:
# holt winters -> exponential smoothing
#additive trend, seasonality
from statsmodels.tsa.holtwinters import ExponentialSmoothing # double and triple exponential smoothing
import pandas as pd
import numpy as np
import warnings
from pmdarima.arima import auto_arima

warnings.simplefilter("ignore")

data = pd.read_csv('C:/Bee Temp Data/three input.csv')

input_cycle = 3 #how manys days to be used as input?
output_cycle = 7 #how manys days to be forecasted ahead?
daily_data_points = 24 #data points per day

In [ ]:
def ARX_forecast():
    forecast_ARX, ARX_error = [], []
    for i in range(0,int(len(data)/daily_data_points),input_cycle+output_cycle):
        print(i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points] #whole dataset -> input_cycle+output_cycle
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        for b in range(output_cycle):
            #train/fitting
            exogenous_train = pd.DataFrame(d_in[0:len(data_in)]) #input exogenous data
            train_ext_temp = exogenous_train['Temp_Box'] - np.mean(exogenous_train['Temp_Box']) #removing offset/trend
            exogenous_ext_temp = np.concatenate((np.linspace(0,len(exogenous_train['Temp_Box'])-1,num=len(exogenous_train['Temp_Box']),dtype = np.uint8).reshape(-1,1),
                                             np.array(train_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['Temp_Box']),-1)
            model = auto_arima( data_in, X = exogenous_ext_temp, start_p = 1, start_q = 0, max_q = 0, information_criterion='aic')
            model.fit(data_in, X = exogenous_ext_temp)
            
            #recursive forecasting
            exogenous_forecast = pd.DataFrame(d_in[(b+input_cycle)*daily_data_points:(b+input_cycle+1)*daily_data_points])
            forecast_ext_temp = exogenous_forecast['Temp_Box'] - np.mean(exogenous_forecast['Temp_Box'])
            exogenous_ext_temp_forecast = np.concatenate((np.linspace(0,len(exogenous_forecast['Temp_Box'])-1,num=len(exogenous_forecast['Temp_Box']),dtype = np.uint8).reshape(-1,1),
                                             np.array(forecast_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Temp_Box']),-1)
            forecast = model.predict(len(exogenous_forecast['Temp_Box']), X = exogenous_ext_temp_forecast)
            data_in= np.concatenate((data_in,forecast))
        forecast_ARX = np.concatenate((forecast_ARX,(data_in[input_cycle*daily_data_points:])))
        ARX_error = np.concatenate((ARX_error,(d_in[input_cycle*daily_data_points:]['Temp'] - data_in[input_cycle*daily_data_points:])**2))
    return forecast_ARX, ARX_error

In [ ]:
def Seasonal_ARX_forecast():
    forecast_sARX, sARX_error = [], []
    for i in range(0,int(len(data)/daily_data_points),input_cycle+output_cycle):
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points] #whole dataset -> input_cycle+output_cycle
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        for b in range(output_cycle):
            #train/fitting
            exogenous_train = pd.DataFrame(d_in[0:len(data_in)]) #input exogenous data
            train_ext_temp = exogenous_train['Temp_Box'] - np.mean(exogenous_train['Temp_Box']) #removing offset/trend
            exogenous_ext_temp = np.concatenate((np.linspace(0,len(exogenous_train['Temp_Box'])-1,num=len(exogenous_train['Temp_Box']),dtype = np.uint8).reshape(-1,1),
                                             np.array(train_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['Temp_Box']),-1)
            model = auto_arima( data_in, X = exogenous_ext_temp, start_p = 1, start_q = 0, max_q = 0, 
                               m = 24, information_criterion='aic')
            model.fit(data_in, X = exogenous_ext_temp)
            
            #recursive forecasting
            exogenous_forecast = pd.DataFrame(d_in[(b+input_cycle)*daily_data_points:(b+input_cycle+1)*daily_data_points])
            forecast_ext_temp = exogenous_forecast['Temp_Box'] - np.mean(exogenous_forecast['Temp_Box'])
            exogenous_ext_temp_forecast = np.concatenate((np.linspace(0,len(exogenous_forecast['Temp_Box'])-1,num=len(exogenous_forecast['Temp_Box']),dtype = np.uint8).reshape(-1,1),
                                             np.array(forecast_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Temp_Box']),-1)
            forecast = model.predict(len(exogenous_forecast['Temp_Box']), X = exogenous_ext_temp_forecast)
            data_in= np.concatenate((data_in,forecast))
        
        forecast_sARX = np.concatenate((forecast_sARX,(data_in[input_cycle*daily_data_points:])))
        sARX_error = np.concatenate((sARX_error,(d_in[input_cycle*daily_data_points:]['Temp'] - data_in[input_cycle*daily_data_points:])**2))
    return forecast_sARX, sARX_error

In [ ]:
def Holt_Winters_forecast():
    forecast_HW, HW_error = [], []
    for i in range(0,int(len(data)/daily_data_points),input_cycle+output_cycle):
        print(int(len(data)/daily_data_points), i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points]
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        for b in range(output_cycle):
            model = ExponentialSmoothing(data_in,
                                         trend='add',
                                         seasonal='add',seasonal_periods=daily_data_points, initialization_method="estimated").fit(optimized = True)
            forecast = model.forecast(daily_data_points)
            #Recursive Forecasting
            data_in= np.concatenate((data_in,forecast))  
        forecast_HW = np.concatenate((forecast_HW,(data_in[input_cycle*daily_data_points:])))
        HW_error = np.concatenate((HW_error,(d_in[input_cycle*daily_data_points:]['Temp'] - data_in[input_cycle*daily_data_points:])**2))
    return forecast_HW, HW_error

In [ ]:
baselines = ['ARX','Seasonal ARX', 'Holt Winters']

column_names = ['ARX', 'ARX error','Seasonal ARX', 'Seasonal ARX error', 'Holt Winters', 'Holt Winters error']
for i in baselines:
    if i == 'ARX':
        forecast_ARX, ARX_error = ARX_forecast()
        all_forecast = pd.DataFrame(np.zeros((len(forecast_ARX), 2*len(baselines))),columns = column_names, index = None)
        all_forecast['ARX'] = np.array(forecast_ARX).reshape(-1,1)
        all_forecast['ARX error'] = np.array(ARX_error).reshape(-1,1)
    elif i == 'Seasonal ARX':
        forecast_sARX, sARX_error =Seasonal_ARX_forecast()
        all_forecast['Seasonal ARX'] = np.array(forecast_sARX).reshape(-1,1)
        all_forecast['Seasonal ARX error'] = np.array(sARX_error).reshape(-1,1)
    elif i == 'Holt Winters':
        forecast_HW, HW_error = Holt_Winters_forecast()
        all_forecast['Holt Winters'] = np.array(forecast_HW).reshape(-1,1)
        all_forecast['Holt Winters error'] = np.array(HW_error).reshape(-1,1)
print(all_forecast)